In [1]:
#Passo1:Importar bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import urllib.parse
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

# --- PASSO 2: Definir a URL e fazer a requisição inicial ---
url = "https://www.farmaponte.com.br/saude/"
url_base = url

# Header para evitar bloqueio
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Criar sessão para reaproveitar conexões
session = requests.Session()
session.headers.update(headers)

# --- Funções auxiliares ---

def achar_nome(div):
    tag_h2 = div.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a:
            return tag_a.get_text(strip=True)
    return "Nome não encontrado"

def achar_preco(div):
    tag_preco = div.find('p', class_="unit-price p-0")
    if tag_preco:
        return tag_preco.get_text().strip()
    return None

def achar_precopix(div):
    tag_precopix = div.find('p', class_="sale-price-pix")
    if tag_precopix:
        strong_tag = tag_precopix.find('strong')
        if strong_tag:
            return strong_tag.get_text().strip()
    return None 

def achar_total_paginas(soup_inicial):
    container_template = soup_inicial.find('div', class_='page-template')
    if container_template:
        divs_candidatas = container_template.find_all('div', class_='text-center pt-3')
        for div in divs_candidatas:
            texto = div.get_text(strip=True)
            if "Página" in texto:
                partes = texto.split()
                ultima_palavra = partes[-1]
                if ultima_palavra.isdigit():
                    return int(ultima_palavra)
    return 1

def achar_link(produto_html, url_base):
    tag_h2 = produto_html.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a and 'href' in tag_a.attrs:
            link_relativo = tag_a.get('href')
            return urllib.parse.urljoin(url_base, link_relativo)
    return "Não encontrado"

def achar_precodesconto(produto_html):
    tag_venda = produto_html.find('p', class_="sale-price p-0")
    if tag_venda:
        strong_tag = tag_venda.find('strong')
        if strong_tag:
            return strong_tag.get_text(strip=True)
    return None

def extrair_detalhes_do_json(soup_detalhes):
    brand = "Não encontrada"
    code = "Não encontrado"
    script_tag = soup_detalhes.find('script', type='application/ld+json')
    if script_tag:
        try:
            raw_json = script_tag.string.strip()
            raw_json = raw_json.replace("\n", "").replace("\r", "").replace("\t", "")
            dados_json = json.loads(raw_json)
            if isinstance(dados_json, list):
                for item in dados_json:
                    if isinstance(item, dict) and 'brand' in item:
                        dados_json = item
                        break
            if 'brand' in dados_json and isinstance(dados_json['brand'], dict):
                brand = dados_json['brand'].get('name', "Não encontrada")
            code = dados_json.get('gtin13', "Não encontrado")
        except Exception as e:
            print("  -> Aviso: Erro ao ler JSON-LD:", e)
    return brand, code

def limpar_preco(preco_str):
    if not preco_str: return None
    try:
        return float(preco_str.replace("R$", "").strip().replace('.', '').replace(',', '.'))
    except (ValueError, TypeError):
        return None

# --- NOVO: baixar página com retry ---
def baixar_url(url, tentativas=3):
    for i in range(tentativas):
        try:
            r = session.get(url, timeout=20)
            if r.status_code == 200:
                return r
        except Exception:
            time.sleep(random.uniform(0.5, 2))
    return None

# --- Função para processar um produto ---
def processar_produto(produto, url_base):
    nome = achar_nome(produto)
    preco = achar_preco(produto)
    precopix = achar_precopix(produto)
    precodesconto = achar_precodesconto(produto)
    link_produto = achar_link(produto, url_base)

    brand, code = None, None
    if link_produto != "Não encontrado":
        responseprodutos = baixar_url(link_produto)
        if responseprodutos:
            soup_produto = BeautifulSoup(responseprodutos.content, 'html.parser')
            brand, code = extrair_detalhes_do_json(soup_produto)

    unit_price = limpar_preco(preco)
    discount_price = limpar_preco(precodesconto)
    pix_price = limpar_preco(precopix)

    discount = round(unit_price - discount_price, 2) if unit_price and discount_price else None
    pix_discount = round(unit_price - pix_price, 2) if unit_price and pix_price else None

    return {
        'Marca': brand,
        'GTIN': code,
        'Desconto': discount,
        'Preco com desconto': discount_price,
        'link': link_produto,
        'Nome': nome,
        'Desconto com pix': pix_discount,
        'Preco pix': pix_price,
        'Preco unitario': unit_price
    }

# --- PRIMEIRA PÁGINA para descobrir total ---
response = baixar_url(url)
soup = BeautifulSoup(response.content, 'html.parser')
total_paginas = achar_total_paginas(soup)

lista_de_produtos = []

# --- LOOP PRINCIPAL ---
for pagina in range(1, total_paginas + 1):
    url_pagina = f'https://www.farmaponte.com.br/saude/?p={pagina}'
    response = baixar_url(url_pagina)
    if not response:
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    div_produtos = soup.find_all('div', class_='li')

    # processa os produtos em paralelo
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(processar_produto, produto, url_base) for produto in div_produtos]
        for future in as_completed(futures):
            resultado = future.result()
            if resultado:
                lista_de_produtos.append(resultado)

    time.sleep(1)  # pequena pausa entre páginas

# --- FINAL ---
print("\nExtração concluída! Gerando tabela...")
df = pd.DataFrame(lista_de_produtos)
display(df)

df.to_csv('farmaponte.csv', index=False)

  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 881 (char 880)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 746 (char 745)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 849 (char 848)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 411 (char 410)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 536 (char 535)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 310 (char 309)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 600 (char 599)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 684 (char 683)
  -> Aviso: Erro ao ler JSON-LD: Expecting ',' delimiter: line 1 column 319 (char 318)

Extração concluída! Gerando tabela...


,Marca,GTIN,Desconto,Preco com desconto,link,Nome,Desconto com pix,Preco pix,Preco unitario
0,Sensaty,7891522022136,None,None,https://www.farmaponte.com.br/fralda-geriatric...,"Fralda Geriatrica Sensaty Premium G, 20 Unidades",None,None,None
1,None,7896004719115,None,None,https://www.farmaponte.com.br/dipirona-gts-10m...,Dipirona Gotas 10ml Germed,None,None,None
2,Sensaty,7891522022143,None,None,https://www.farmaponte.com.br/fralda-geriatric...,"Fralda Geriatrica Sensaty Premium Eg, 18 Unidades",None,None,None
3,Sensaty,7891522022129,None,None,https://www.farmaponte.com.br/fralda-geriatric...,"Fralda Geriatrica Sensaty Premium M, 20 Unidades",None,None,None
4,Eurofarma,7891317127800,None,None,https://www.farmaponte.com.br/tadalafila-eurof...,"Tadalafila Eurofarma 20mg, Caixa Com 4 Comprim...",None,None,None
...,...,...,...,...,...,...,...,...,...
6508,None,7896472521777,None,None,https://www.farmaponte.com.br/valsartana-160mg...,Valsartana 160mg Genérico Nova Química 30 Comp...,None,None,None
6509,None,7899095204072,None,None,https://www.farmaponte.com.br/oftalmox-cloridr...,Oftalmox Cloridrato De Moxifloxacino 5mg Ml So...,None,None,None
6510,None,7896714291437,None,None,https://www.farmaponte.com.br/bilastina-20mg-n...,"Bilastina 20mg Neo Quimica, Caixa Com 30 Compr...",None,None,None
6511,None,7897076922519,None,None,https://www.farmaponte.com.br/amoxicilina-500m...,Amoxicilina 500mg + Clavulanato De Potássio 12...,None,None,None
